In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.metrics import mean_squared_error as mse

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

from IPython.display import display, HTML

import eli5

from pyastrotrader import calculate_chart, calculate_aspects, calculate_transits
from pyastrotrader.utils import create_input_json
from pyastrotrader.constants import *

from settings import *
from helpers import *

In [ ]:
today = datetime.datetime.now().date()
days_to_process = []
data_to_dataframe = []
day = today
for day_to_predict in range(DAYS_TO_PREDICT):
    day = day + datetime.timedelta(days=1)
    day_as_str = day.strftime('%Y-%m-%d')
    days_to_process.append(day_as_str)
    data_to_dataframe.append({ 'CorrectedDate' : day_as_str})
    
    

In [ ]:
StockPrices = pd.DataFrame(data_to_dataframe)

In [ ]:
asset_natal_chart_input = create_input_json(NATAL_DATE, 
                                            DEFAULT_PARAMETERS, 
                                            DEFAULT_CONFIG)

asset_natal_chart = calculate_chart(asset_natal_chart_input)
dates_to_generate = days_to_process

In [ ]:
for current_date in dates_to_generate:
    chart_input = create_input_json(current_date + 'T10:00:00-03:00', 
                                      DEFAULT_PARAMETERS, 
                                      DEFAULT_CONFIG)
    charts[current_date] = calculate_chart(chart_input)
    aspects[current_date] = calculate_transits(asset_natal_chart, charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)
    aspects_transiting[current_date]= calculate_aspects(charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)   

In [ ]:
astro_columns = []

for first_planet in PLANETS_TO_CALCULATE:
    for second_planet in PLANETS_TO_CALCULATE:
        for aspect in ASPECTS_TO_CALCULATE:
            column_name="ASTRO_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')
            
            if first_planet == second_planet:
                continue
                
            column_name="ASTRO_TRANSITING_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected_transiting(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')                 

In [ ]:
for first_planet in PLANETS_TO_CALCULATE:
    column_name="ASTRO_{}_RETROGADE".format(PLANETS[first_planet]).upper()
    astro_columns.append(column_name)
    StockPrices[column_name] = StockPrices.apply(lambda x:is_retrograde(x, first_planet), axis =1)
    StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float',errors='coerce')

In [ ]:
booster_swing_trade = xgb.Booster()
booster_price_increase = xgb.Booster()
booster_price_decrease = xgb.Booster()
booster_price_stagnation = xgb.Booster()

swing_trade_model_filename = './output/{}_swing_trade.model'.format(ASSET_TO_CALCULATE)
price_increase_model_filename = './output/{}_price_increase.model'.format(ASSET_TO_CALCULATE)
price_decrease_model_filename = './output/{}_price_decrease.model'.format(ASSET_TO_CALCULATE)
price_stagnation_model_filename = './output/{}_price_stagnation.model'.format(ASSET_TO_CALCULATE)

booster_swing_trade.load_model(swing_trade_model_filename)  # load data
booster_price_increase.load_model(price_increase_model_filename)  # load data
booster_price_decrease.load_model(price_decrease_model_filename)  # load data
booster_price_stagnation.load_model(price_stagnation_model_filename)  # load data

In [ ]:
StockPrices['PredictSwingTradeScore'] = StockPrices.apply(lambda x:predict_score(x, booster_swing_trade, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceIncreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_increase, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceDecreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_decrease, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceStagnation'] = StockPrices.apply(lambda x:predict_score(x, booster_price_stagnation, StockPrices, astro_columns), axis =1)

In [ ]:
SimplifiedColumns = ['CorrectedDate', 'PredictSwingTradeScore', 'PredictPriceIncreaseScore','PredictPriceDecreaseScore','PredictPriceStagnation']
#SimplifiedColumns

In [ ]:
output_excel_file='./output/{}.Predict.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)

output_excel_file='./output/{}.Predict.Simplified.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices[SimplifiedColumns].to_excel(output_excel_file)